<a href="https://colab.research.google.com/github/vladgap/LLC/blob/main/Rotem%20LLC%20Extraction%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from sklearn import preprocessing
import matplotlib.pyplot as plt
pd.options.plotting.backend = "plotly"

In [23]:
!wget -O MLNN.py "https://raw.githubusercontent.com/vladgap/MLNN/main/MLNN_3_2.py"
from MLNN import *

--2025-02-12 07:26:59--  https://raw.githubusercontent.com/vladgap/MLNN/main/MLNN_3_2.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14169 (14K) [text/plain]
Saving to: ‘MLNN.py’

MLNN.py             100%[===================>]  13.84K  --.-KB/s    in 0s      

2025-02-12 07:26:59 (61.0 MB/s) - ‘MLNN.py’ saved [14169/14169]



In [24]:
!wget -O LLC.py "https://raw.githubusercontent.com/vladgap/LLC/main/LLC_versions/LLC_5_2.py"
import LLC

--2025-02-12 07:26:59--  https://raw.githubusercontent.com/vladgap/LLC/main/LLC_versions/LLC_5_2.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44327 (43K) [text/plain]
Saving to: ‘LLC.py’

LLC.py              100%[===================>]  43.29K  --.-KB/s    in 0.01s   

2025-02-12 07:26:59 (3.74 MB/s) - ‘LLC.py’ saved [44327/44327]



In [25]:
# import LLC_5_2 as LLC52

In [26]:
def CopyPasteToArray(a):
  if a.startswith('\n'):
    b=a[1:]
  else:
    b=a
  if b.endswith('\n'):
    b=b[:-1]
  c=b.replace('\t',',')
  d=c.split('\n')
  f=[]
  for e in d:
    if e.replace(',','').replace('.','').isdigit(): # only digits no letters
      f.append(list(eval(e)))
    else:
      f.append(e.split(','))
  return f

# EQUIL data

## EQUIL_rotem_extr1
data 8 -- EQUIL_rotem_extraction 2+3

In [27]:
# X: 'H3PO4_aq'	'SO4_aq'
# T: 'FA_aq'	'solvent_aq'	'H3PO4_org'	'SO4_org'	'water_org'

scaler_data=[[37.23870967741936, 7.199806451612903],
 [21.87534593155935, 6.51727589252697],
 [155.77096774193546,
  0.5283870967741936,
  13.459354838709675,
  1.0374193548387098,
  7.283870967741936],
 [95.42174024805198,
  0.5151599612158911,
  13.221703793258648,
  1.1486934169207217,
  1.7674652479484612]]

scaler_X = preprocessing.StandardScaler().fit([[0,0]])
scaler_T1 = preprocessing.StandardScaler().fit([[0]])
scaler_T2 = preprocessing.StandardScaler().fit([[0]])
scaler_T3 = preprocessing.StandardScaler().fit([[0]])
scaler_T4 = preprocessing.StandardScaler().fit([[0]])

scaler_X.mean_, scaler_X.scale_ = scaler_data[0], scaler_data[1]
scaler_T1.mean_, scaler_T1.scale_ = scaler_data[2][1], scaler_data[3][1] # solvent_aq
scaler_T2.mean_, scaler_T2.scale_ = scaler_data[2][2], scaler_data[3][2] # H3PO4_org
scaler_T3.mean_, scaler_T3.scale_ = scaler_data[2][3], scaler_data[3][3] # SO4_org
scaler_T4.mean_, scaler_T4.scale_ = scaler_data[2][4], scaler_data[3][4] # water_org


# scaler_T1 = preprocessing.StandardScaler().fit([[0]])
# scaler_T2 = preprocessing.StandardScaler().fit([[0]])
# scaler_T3 = preprocessing.StandardScaler().fit([[0]])
# scaler_T3_4 = preprocessing.StandardScaler().fit([[0]])

# scaler_X3.mean_, scaler_X3.scale_ = Xscale[0], Xscale[1]
# scaler_X3_2.mean_, scaler_X3_2.scale_ = [27.03608333,  0.0474    ], [14.68499343,  0.04213688] # מים בקלה חושבו רק על נתונים "טובים
# scaler_T3_1.mean_, scaler_T3_1.scale_=[Tscale[0][0]], [Tscale[1][0]]
# # scaler_T3_2.mean_, scaler_T3_2.scale_=[Tscale[0][1]], [Tscale[1][1]]
# scaler_T3_2.mean_, scaler_T3_2.scale_=[10.81], [1.3091664] # מים בקלה חושבו רק על נתונים "טובים
# scaler_T3_3.mean_, scaler_T3_3.scale_=[Tscale[0][2]], [Tscale[1][2]]
# scaler_T3_4.mean_, scaler_T3_4.scale_=[Tscale[0][3]], [Tscale[1][3]]

In [28]:
class EQUIL_rotem_extr1: #
# data8
# gets xout [%solvent_aq, %H3PO4_aq, %SO4_aq]
# calc xout , yout [%water_org, %H3PO4_org, %SO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], ...]
        # the model predicts H3PO4_org, water_org ,solvent_aq, SO4_org
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        # x[:,[2]] is for %SO4_aq

        # scaler
        scaler_data=[[37.23870967741936, 7.199806451612903],
        [21.87534593155935, 6.51727589252697],
        [155.77096774193546,
          0.5283870967741936,
          13.459354838709675,
          1.0374193548387098,
          7.283870967741936],
        [95.42174024805198,
          0.5151599612158911,
          13.221703793258648,
          1.1486934169207217,
          1.7674652479484612]]

        scaler_X = preprocessing.StandardScaler().fit([[0,0]])
        scaler_T1 = preprocessing.StandardScaler().fit([[0]]) # solvent_aq
        scaler_T2 = preprocessing.StandardScaler().fit([[0]]) # H3PO4_org
        scaler_T3 = preprocessing.StandardScaler().fit([[0]]) # SO4_org
        scaler_T4 = preprocessing.StandardScaler().fit([[0]]) # water_org

        scaler_X.mean_, scaler_X.scale_ = scaler_data[0], scaler_data[1]
        scaler_T1.mean_, scaler_T1.scale_ = scaler_data[2][1], scaler_data[3][1] # solvent_aq
        scaler_T2.mean_, scaler_T2.scale_ = scaler_data[2][2], scaler_data[3][2] # H3PO4_org
        scaler_T3.mean_, scaler_T3.scale_ = scaler_data[2][3], scaler_data[3][3] # SO4_org
        scaler_T4.mean_, scaler_T4.scale_ = scaler_data[2][4], scaler_data[3][4] # water_org

        X_scaled=scaler_X.transform(x[:,[1,2]])
        X_scaled=np.clip(X_scaled, -2, 2)

        # H3PO4_org
        bp1=VectorBackProp(layers=[2,1,1], hidden_activation = 'sigmoid')
        manual_weights1=[[[-2.2542856480833247], [-0.7115892162097907]],
 [[2.11693061595761]],
 [[-4.9502436032281585]],
 [[3.912054301757645]]]
        bp1.import_weights(manual_weights1)
        predics1=scaler_T2.inverse_transform(bp1.run(X_scaled))

        # water_org
        bp2=VectorBackProp(layers=[2,2,1], hidden_activation = 'sigmoid')
        manual_weights2=[[[3.865511470369896, 8.917531695210526],
  [2.5196000386044317, 1.9258172086316168]],
 [[-2.2766428706488764, -1.5079705074058127]],
 [[2.7771600466732367], [0.9281118411212552]],
 [[-1.1710007634929602]]]
        bp2.import_weights(manual_weights2)
        predics2=scaler_T4.inverse_transform(bp2.run(X_scaled))

        # solvent_aq
        bp3=VectorBackProp(layers=[2,1,1], hidden_activation = 'linear')
        manual_weights3=[[[0], [0]], [[0]], [[0]], [[0.9]]]
        bp3.import_weights(manual_weights3)
        predics3=scaler_T1.inverse_transform(bp3.run(X_scaled))

        # SO4_org
        bp4=VectorBackProp(layers=[2,3,1], hidden_activation = 'sigmoid')
        manual_weights4=[[[5.255129630698163, 2.5370572590919305, -2.771603241282131],
  [4.30266945368854, 1.6643297587562575, 2.759812469181559]],
 [[-6.720893024720382, -0.7893443670545283, 5.835081175805133]],
 [[5.160798955335814], [2.555891510916236], [2.30456660012357]],
 [[-3.1529870476605937]]]
        bp4.import_weights(manual_weights4)
        predics4=scaler_T3.inverse_transform(bp4.run(X_scaled))

        self.xout = np.array([predics3[:,0], x[:,1], x[:,2]]).T # [[%solvent_aq, %H3PO4_aq, %SO4_aq], ...]
        self.yout = np.array([predics2[:,0], predics1[:,0], predics4[:,0]]).T # [[%water_org, %H3PO4_org, %SO4_org], ...]

        # print (self.yout, self.xout)

# Cases

## Extraction 1

### test

In [29]:
x=np.array([[0,0,0],[100,100,100]])

In [30]:
X_scaled=scaler_X.transform(x[:,[1,2]])
X_scaled

array([[-1.70231409, -1.10472636],
       [ 2.86904219, 14.23910773]])

In [31]:
np.clip(X_scaled, -1, 1)

array([[-1., -1.],
       [ 1.,  1.]])

In [32]:
EQUIL_rotem_extr1([[0,30,9]]).xout # [[%solvent_aq, %H3PO4_aq, %SO4_aq], ...]

array([[ 0.99203106, 30.        ,  9.        ]])

In [33]:
EQUIL_rotem_extr1([[0,30,9]]).yout # [[%water_org, %H3PO4_org, %SO4_org], ...]

array([[5.51179427, 3.98631405, 0.76135935]])

In [34]:
test1 = LLC.Stage(173.2, 15, [8.2, .06, .06], [1, 30, 15], EQUIL_rotem_extr1, eff=.8, convergence=1E-4)#, entrainment_perc=[1,2,3,4], entrainment_comp_in=[[10,10],[5,5]])
test1.runs

12

In [35]:
test1.output()

Oout= 169.57346101275468
yout= [5.25893594 0.85771993 0.26318594]
yout_tag= [5.25893594 1.05714991 0.31398242]  %water_org_equilib, %solubles_equilib
Aout= 18.626538987245322
xout= [ 0.99203106 16.90836857 10.24168195]
equilibrium efficiency= [1.  0.8 0.8]
error= [-3.23113626e-07  7.40483595e-06 -9.94591222e-05]
convergence= [-3.11837121e-06 -7.43013235e-05  1.02388794e-05]


In [36]:
test2 = LLC.Stage(173.2, 50, [6, 6, 1], [1, 60, .1], EQUIL_rotem_extr1, eff=.8, convergence=1E-4)#, entrainment_perc=[1,2,3,4], entrainment_comp_in=[[10,10],[5,5]])
test2.runs

61

In [37]:
test2.output()

Oout= 188.48554843890858
yout= [ 7.27413364 12.0435486   0.695869  ]
yout_tag= [ 7.27413364 13.55443574  0.61983625]  %water_org_equilib, %solubles_equilib
Aout= 34.7144515610914
xout= [ 0.99203106 51.17669027  1.36347826]
equilibrium efficiency= [1.  0.8 0.8]
error= [ 1.94927193e-05 -3.26232763e-03 -2.23884450e-03]
convergence= [-4.70070399e-05  3.24166294e-05  8.06081543e-05]


In [38]:
# test3 = LLC52.Stage(173.2, 50, [6, 6, .5], [1, 60, .1], EQUIL_rotem_extr1, eff=.8, convergence=1E-4)

### mesh

In [39]:
# x_mesh, y_mesh = np.linspace(0, 60, 31), np.linspace(0, 0.2, 31) # expanded x and y extrapolation area observation
x_mesh, y_mesh = np.linspace(0, 70, 31), np.linspace(0, 25, 31)
x_mesh, y_mesh = np.meshgrid(x_mesh, y_mesh)
x_mesh3 = x_mesh.flatten().reshape(31*31,1)
y_mesh3 = y_mesh.flatten().reshape(31*31,1)
mesh3=np.concatenate((x_mesh3, y_mesh3), axis=1)
# x_mesh[:5]

In [40]:
a=np.concatenate((np.zeros([len(mesh3),1]),mesh3), axis=1)

In [41]:
mesh_predict_x=np.array([EQUIL_rotem_extr1([row]).xout for row in a])
mesh_predict_y=np.array([EQUIL_rotem_extr1([row]).yout for row in a])
mesh_predics1_1=mesh_predict_y[:,:,1]
mesh_predics1_2=mesh_predict_y[:,:,0]
mesh_predics1_3=mesh_predict_x[:,:,0]
mesh_predics1_4=mesh_predict_y[:,:,2]

### 1 - Extraction1 - raffinate

In [59]:
# bat1_1=LLC.Battery(stages_num=3, Oin=173.2, Ain=15, yin=[8.2, .06, .06], xin=[1, 30, 15], EQUIL=EQUIL_rotem_extr1, eff=.8, max_iter=200)#, convergence=1E-4)
bat1_1=LLC.Battery(stages_num=5, Oin=173.2, Ain=50, yin=[6, 6, 1 ], xin=[1, 60, .1], EQUIL=EQUIL_rotem_extr1, method='successive', eff=.8, damping=.5, max_iter=50, battery_convergence=0.03, stage_convergence=1e-5)#, entrainment_perc_in=[1,2], entrainment_comp_in=[[10,10,10],[5,5,5]], entrainment_perc_out=[3,4])
bat1_1.runs

23

In [60]:
bat1_1.errors_array.plot()

In [52]:
bat1_1.errors_array.plot()

In [61]:
table1_1=LLC.BatteryTable(bat1_1).data

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5
"Org in, ton/hr",173.200000,174.304069,178.167703,181.335957,187.813128
"y1 in, %",6.000000,5.958976,6.364595,6.719503,7.081885
"y2 in, %",6.000000,6.836593,8.146627,9.589359,11.776794
"y3 in, %",1.000000,0.745645,0.617343,0.528333,0.451992
"Aq in, ton/hr",29.239372,32.820291,36.182281,41.022670,50.000000
"x1 in, %",0.992031,0.992031,0.992031,0.992031,1.000000
"x2 in, %",45.002399,47.611867,50.553777,54.397890,60.000000
"x3 in, %",2.132192,1.355684,0.845729,0.459249,0.100000
"Org out, ton/hr",174.309983,177.166709,181.663114,186.499346,197.042483
"y1 out, %",5.958742,6.310905,6.697616,7.060843,7.454669


In [45]:
# @markdown plots
fig = make_subplots(rows=2, cols=2, subplot_titles=('H3PO4_org','water_org','solvent_aq','SO4_org'), column_widths=[0.5, 0.5], vertical_spacing=0.05,
                     specs=[[{"type": "scene"}, {"type": "scene"}],[{"type": "scene"}, {"type": "scene"}]])

# visib='legendonly'
visib=True
fig.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics1_1[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,1)
fig.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics1_2[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,2)
fig.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics1_3[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,1)
fig.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics1_4[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,2)

bat=bat1_1
# n, row, col =[1,0,2], [1,1,2], [1,2,2] # H3PO4_org, water_org, SO4_org
# for i in range(3):
#   fig.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,n[i]], mode='markers', marker_color='blue', name='Efficiency'), row[i], col[i])
#   x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,n[i]],[bat.yout_list[-1,n[i]]]))
#   fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),row[i],col[i])

# H3PO4_org
fig.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,1], mode='markers', marker_color='blue', name='Efficiency'), 1, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,1],[bat.yout_list[-1,1]]))
fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,1)
# water_org
fig.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 1, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,0],[bat.yout_list[-1,0]]))
fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,2)
# SO4_org
fig.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,2], mode='markers', marker_color='blue', name='Efficiency'), 2, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,2],[bat.yout_list[-1,2]]))
fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,2)

# solvent_aq
fig.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.xout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 2, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.xout_list[:,0],[bat.xin_list[-1,0]])) # aqueous -- different operation line order
fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,1)


fig.update_layout(title='', autosize=True,
                  width=980, height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig.update_layout(
    # scene1_zaxis_range=[0,10],
    # scene2_zaxis_range=[10,14],
    # scene3_zaxis_range=[0,3],
    # scene4_zaxis_range=[0,.1],
    )

# fig.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=1.5, y=-1.5, z=1.5))
fig.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=0, y=-2.2, z=0.01), aspectratio=dict(x=1, y=1, z=1))
fig.update_scenes(camera_projection_type="orthographic")

fig.show()